## R2 To R1 Point Distance Attractor, S1 Damping

In [ ]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools

## Setup

### Point Distance Attractor

In [ ]:
PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{R2,R1}}, CN::Chart{1,R1}) = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{R2,R1}}, CN::Chart{1,R1}) = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R2,R1}}, CN::Chart{1,R1}) = 
    -3*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R2,R1}}, CN::Chart{1,R1}) = 
    default_weight_metric(xn, vn, task, CN)

### Angular Damping Around Goal

In [ ]:
PBDS.default_coord_rep(::Damping{<:AngularPositionAroundPoint{R2,S1}}) = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:AngularPositionAroundPoint{R2,S1}}) =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:AngularPositionAroundPoint{R2,S1}}) = 0.
PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R2,S1}}) = -5*vne
PBDS.weight_metric_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R2,S1}}) =
    default_weight_metric(xne, vne, task)

In [ ]:
xm_goal = SA[0., 0.]
M = R2
CM = Chart{1,M}()
tasks, CNs = TaskList(), ChartList()

N = R1
push!(tasks, Attractor(DistanceFromPoint{M,N}(xm_goal)))
push!(CNs, Chart{1,N}())

N = S1
push!(tasks, Damping(AngularPositionAroundPoint{M,N}(xm_goal)))
push!(CNs, Chart{Angleπ,N}())

## Point Acceleration

In [ ]:
# Initial state
xm = SA[-1., 1.]
vm = SA[1., 1.]
σxddot, = multiple_task_acceleration(xm, vm, tasks, CM, CNs)

## Single Trajectory

In [ ]:
Time = 10.
dt = 0.01

traj = propagate_tasks(xm, vm, tasks, CM, CNs, Time, dt)
traj.xm[end]

## Multiple Trajectories

In [ ]:
using Makie, Observables, ProgressMeter, VectorizedRoutines

In [ ]:
Time = 5
dt = 0.02
xlim, Δx = 1., 1.
vlim, Δv = 1., 2.

xm0 = Array{Any}(undef, 2)
vm0 = Array{Any}(undef, 2)

xm0[1], xm0[2] = Matlab.meshgrid(-xlim:Δx:xlim, -xlim:Δx:xlim)
vm0[1], vm0[2] = Matlab.meshgrid(-vlim:Δv:vlim, -vlim:Δv:vlim)

for a in (xm0, vm0), i in 1:2
    a[i] = reshape(a[i], length(a[i]))
end

nx = length(xm0[1])
nv = length(vm0[1])
ix, iv = Matlab.meshgrid(1:nx, 1:nv)
ix = reshape(ix, length(ix))
iv = reshape(iv, length(iv))

ntraj = nx*nv
trajs = Array{Any}(undef, ntraj)
xm0 = [xm0[1]'; xm0[2]']
vm0 = [vm0[1]'; vm0[2]']
p = Progress(ntraj)

Threads.@threads for i in 1:ntraj
    trajs[i] = propagate_tasks(SA[xm0[:,ix[i]]...], SA[vm0[:,iv[i]]...], tasks, CM, CNs, Time, dt)
    next!(p)
end

In [ ]:
fxx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fxy(i) = [trajs[j].xm[i][2] for j = 1:ntraj]
iobs = Observable(1)

ax_size, plot_size = 2, 500
limits = FRect((-ax_size, -ax_size), (2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fxx(i), iobs), lift(i -> fxy(i), iobs), markersize = ax_size/20, color = :blue, limits = limits)
Makie.scatter!(scene, xm_goal', markersize = ax_size/20, color = :darkgreen)
for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), color = :purple)
end

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
display(scene)

In [ ]:
function record_scene(scene, filename, iobs, N)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        next!(p)
    end
    display("text/html", html_video(filename))
end

filename = "R2ToR1PointAttractor_S1Damping.mp4"
record_scene(scene, filename, iobs, length(trajs[1].xm))